In [1]:
import os
import math
import copy

import numpy as np
import pickle
import networkx as nx
import pandas as pd

import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import QM9
import matplotlib.pyplot as plt
from torch_geometric.utils import from_networkx, to_networkx, to_undirected, to_scipy_sparse_matrix
from torch_geometric.data.data import Data
from torch_geometric.loader import DataLoader

In [2]:
qm9 = QM9(root='dataset/QM9')

/home/lizhifei/.local/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _

In [3]:
y_target = pd.DataFrame(qm9.data.y.numpy())

qm9 = qm9.shuffle()

/home/lizhifei/.local/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
# data split
data_size = 30000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)

In [7]:
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

In [8]:
aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import PNAConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class PNANet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr, aggregators, scalers, deg):
        super(PNANet, self).__init__()
        
        # Define PNA layers with specified aggregators, scalers, and degree tensor
        self.conv1 = PNAConv(
            in_channels=num_node_features,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv2 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv3 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        
        # Define linear layers for final graph-level output
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Pass node features and edge attributes through PNA layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

# Target 0

In [10]:
target = 0
qm9.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9.data.y[0:train_index].mean()
data_std = qm9.data.y[0:train_index].std()

qm9.data.y = (qm9.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9[test_index:val_index], batch_size=64, shuffle=True)

/home/lizhifei/.local/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9.num_features
dim_h = 64
edge_attr = qm9[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [22]:
def training(loader, model, loss, optimizer):
    """Training one epoch

    Args:
        loader (DataLoader): loader (DataLoader): training data divided into batches
        model (nn.Module): GNN model to train on
        loss (nn.functional): loss function to use during training
        optimizer (torch.optim): optimizer during training

    Returns:
        float: training loss
    """
    model.train()

    current_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        data.x = data.x.float()

        out = model(data)

        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        current_loss += l / len(loader)
        l.backward()
        optimizer.step()
    return current_loss, model

In [23]:
def validation(loader, model, loss):
    """Validation

    Args:
        loader (DataLoader): validation set in batches
        model (nn.Module): current trained model
        loss (nn.functional): loss function

    Returns:
        float: validation loss
    """
    model.eval()
    val_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        val_loss += l / len(loader)
    return val_loss

In [24]:
@torch.no_grad()
def testing(loader, model):
    """Testing

    Args:
        loader (DataLoader): test dataset
        model (nn.Module): trained model

    Returns:
        float: test loss
    """
    loss = torch.nn.MSELoss()
    test_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        # NOTE
        # out = out.view(d.y.size())
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        test_loss += l / len(loader)


    return test_loss

In [27]:
def train_epochs(epochs, model, train_loader, val_loader, path):
    """Training over all epochs

    Args:
        epochs (int): number of epochs to train for
        model (nn.Module): the current model
        train_loader (DataLoader): training data in batches
        val_loader (DataLoader): validation data in batches
        path (string): path to save the best model

    Returns:
        array: returning train and validation losses over all epochs, prediction and ground truth values for training data in the last epoch
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    loss = torch.nn.L1Loss()

    train_loss = np.empty(epochs)
    val_loss = np.empty(epochs)
    best_loss = math.inf

    for epoch in range(1, epochs):
        epoch_loss, model = training(train_loader, model, loss, optimizer)
        v_loss = validation(val_loader, model, loss)
        if v_loss < best_loss:
            best_loss = v_loss
            torch.save(model.state_dict(), path)
        for data in train_loader:
            data = data.to(device)
            out = model(data)

        train_loss[epoch] = epoch_loss.detach().cpu().numpy()
        val_loss[epoch] = v_loss.detach().cpu().numpy()

        # print current train and val loss
        if epoch % 5 == 0:
            print(
                "Epoch: "
                + str(epoch)
                + ", Train loss: "
                + str(epoch_loss.item())
                + ", Val loss: "
                + str(v_loss.item())
            )
    return best_loss, train_loss, val_loss

In [28]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model.pt"
)

Epoch: 5, Train loss: 0.4855073392391205, Val loss: 0.45234110951423645
Epoch: 10, Train loss: 0.48221397399902344, Val loss: 0.4539158344268799
Epoch: 15, Train loss: 0.4730902910232544, Val loss: 0.4520643651485443
Epoch: 20, Train loss: 0.46881183981895447, Val loss: 0.47190800309181213
Epoch: 25, Train loss: 0.4659658670425415, Val loss: 0.4571883976459503
Epoch: 30, Train loss: 0.46022576093673706, Val loss: 0.47000473737716675
Epoch: 35, Train loss: 0.45941585302352905, Val loss: 0.431653767824173
Epoch: 40, Train loss: 0.45958787202835083, Val loss: 0.4693063497543335
Epoch: 45, Train loss: 0.45121821761131287, Val loss: 0.45378926396369934
Epoch: 50, Train loss: 0.44882744550704956, Val loss: 0.4332180917263031
Epoch: 55, Train loss: 0.44909578561782837, Val loss: 0.4616988003253937
Epoch: 60, Train loss: 0.44400152564048767, Val loss: 0.43069541454315186
Epoch: 65, Train loss: 0.44751715660095215, Val loss: 0.43895646929740906
Epoch: 70, Train loss: 0.4426189064979553, Val los

In [29]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/3980023614.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model.pt"))


Test Loss for GIN: 0.34658658504486084


## Extra Node

In [52]:
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import torch

def networkx_to_pyg_with_attributes(G):
    # Convert NetworkX graph to PyG Data object with node and edge attributes
    data = from_networkx(G, group_node_attrs=['x'], group_edge_attrs=['edge_attr'])

    # If there are global attributes stored in G.graph, add them back to the PyG Data object
    if 'y' in G.graph:
        data.y = torch.tensor(G.graph['y']) if isinstance(G.graph['y'], list) else G.graph['y']
    
    return data


In [53]:
import torch
from torch_geometric.utils import to_networkx
import networkx as nx

def pyg_to_networkx_with_attributes(data):
    # Convert PyG data to NetworkX graph, retaining node attributes
    G = to_networkx(data, node_attrs=['x'], edge_attrs=['edge_attr'])

    # Add global attributes manually (if any exist)
    G.graph['y'] = data.y

    return G


In [62]:
import torch
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

def add_extra_node_on_each_edge(data):
    # Convert PyG data to NetworkX graph for easier manipulation
    G = pyg_to_networkx_with_attributes(data)
    
    # Original number of nodes
    num_original_nodes = G.number_of_nodes()
    
    # Prepare lists for new features
    edges = list(G.edges(data=True))
    new_node_features = []
    new_edges_src = []
    new_edges_dst = []
    new_edge_features = []
    original_edge_attrs = list(data.edge_attr)  # Store original edge attributes

    # Keep track of removed edges to exclude their attributes
    removed_edge_indices = []

    for idx, (u, v, edge_data) in enumerate(edges):
        # Remove the original edge and track its index for exclusion
        G.remove_edge(u, v)
        removed_edge_indices.append(idx)

        # Create new node as the mean of connected node features
        new_node_id = num_original_nodes + len(new_node_features)
        new_node_feature = (data.x[u] + data.x[v]) / 2
        new_node_features.append(new_node_feature)
        
        # Add new node with feature
        G.add_node(new_node_id, x=new_node_feature)

        # Add edges from new node to each original node
        G.add_edge(u, new_node_id)
        G.add_edge(new_node_id, v)

        # Use original edge feature for each new edge
        edge_feature = edge_data['edge_attr']
        edge_feature_tensor = (
            edge_feature if isinstance(edge_feature, torch.Tensor) else torch.tensor(edge_feature)
        )
        new_edge_features.append(edge_feature_tensor)  # for edge (u, new_node_id)
        new_edge_features.append(edge_feature_tensor)  # for edge (new_node_id, v)
    
    # Filter out removed edges from the original edge_attr
    kept_edge_attr = [attr for i, attr in enumerate(original_edge_attrs) if i not in removed_edge_indices]

    # Convert back to PyG Data object
    modified_data = from_networkx(G)

    # Update node features
    modified_data.x = torch.cat([data.x, torch.stack(new_node_features)], dim=0)

    # Update edge features: combine kept edge features with new edge features
    modified_data.edge_attr = torch.cat([torch.stack(kept_edge_attr), torch.stack(new_edge_features)], dim=0)

    modified_data.y = data.y
    
    return modified_data



In [63]:
import copy
# Apply the transformation across a subset of QM9
qm9_ExN = []
ExN_set = copy.deepcopy(qm9[:30000])
for data in ExN_set:  # Process 100 molecules as an example
    modified_data = add_extra_node_on_each_edge(data)
    qm9_ExN.append(modified_data)

RuntimeError: stack expects a non-empty TensorList

In [60]:
ExN_set[0]

Data(x=[19, 11], edge_index=[2, 36], edge_attr=[36, 4], y=[1], pos=[19, 3], z=[19], smiles='[H]C(=O)OC([H])([H])C([H])([H])[C@@]([H])(C([H])=O)C([H])([H])[H]', name='gdb_62256', idx=[1])

In [61]:
qm9_ExN[0]

Data(x=[55, 11], edge_index=[2, 72], y=[1], edge_attr=[108, 4])

## Subgraph Extraction

In [64]:
def extract_local_subgraph_features_with_edges(data, radius=2):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Initialize lists to store node and edge features
    subgraph_sizes = []
    subgraph_degrees = []
    edge_subgraph_centralities = []

    # Compute node-level subgraph features
    for node in G.nodes():
        subgraph = nx.ego_graph(G, node, radius=radius)
        subgraph_size = subgraph.number_of_nodes()
        subgraph_degree = np.mean([d for n, d in subgraph.degree()])

        subgraph_sizes.append(subgraph_size)
        subgraph_degrees.append(subgraph_degree)

    # Convert node features to tensors
    subgraph_sizes_tensor = torch.tensor(subgraph_sizes, dtype=torch.float).view(-1, 1)
    subgraph_degrees_tensor = torch.tensor(subgraph_degrees, dtype=torch.float).view(-1, 1)
    data.x = torch.cat([data.x, subgraph_sizes_tensor, subgraph_degrees_tensor], dim=-1)

    # Compute edge-level subgraph features
    for u, v in G.edges():
        subgraph_u = nx.ego_graph(G, u, radius=radius)
        edge_centrality = nx.edge_betweenness_centrality(subgraph_u, normalized=True).get((u, v), 0)
        edge_subgraph_centralities.append(edge_centrality)

    # Convert edge features to tensor and concatenate to edge_attr
    edge_centrality_tensor = torch.tensor(edge_subgraph_centralities, dtype=torch.float).view(-1, 1)
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data


In [65]:
import copy
qm9_data = qm9[:30000]
qm9_sub = copy.deepcopy(qm9_data)
qm9_SE = []
for data in qm9_sub:
    data_sub = extract_local_subgraph_features_with_edges(data, radius=2)
    qm9_SE.append(data_sub)
    

In [68]:
# datasets into DataLoader
train_loader = DataLoader(qm9_SE[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_SE[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_SE[test_index:val_index], batch_size=64, shuffle=True)

In [69]:
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_SE[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_SE[0].x.shape[1]
dim_h = 64
edge_attr = qm9_SE[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_SE[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [71]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_SE.pt"
)

Epoch: 5, Train loss: 0.5388436317443848, Val loss: 0.47594335675239563
Epoch: 10, Train loss: 0.502517580986023, Val loss: 0.481692373752594
Epoch: 15, Train loss: 0.4866790473461151, Val loss: 0.4826018214225769
Epoch: 20, Train loss: 0.4694737493991852, Val loss: 0.4423438608646393
Epoch: 25, Train loss: 0.467204213142395, Val loss: 0.44274091720581055
Epoch: 30, Train loss: 0.45695260167121887, Val loss: 0.435397207736969
Epoch: 35, Train loss: 0.45430871844291687, Val loss: 0.43345534801483154
Epoch: 40, Train loss: 0.448910117149353, Val loss: 0.44769564270973206
Epoch: 45, Train loss: 0.4464457631111145, Val loss: 0.4617311358451843
Epoch: 50, Train loss: 0.44150856137275696, Val loss: 0.44321370124816895
Epoch: 55, Train loss: 0.4415639638900757, Val loss: 0.4282848536968231
Epoch: 60, Train loss: 0.43270841240882874, Val loss: 0.41854554414749146
Epoch: 65, Train loss: 0.4346553385257721, Val loss: 0.43637746572494507
Epoch: 70, Train loss: 0.43330496549606323, Val loss: 0.421

In [75]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_SE.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))

ZeroDivisionError: float division by zero

## Virtual Node

In [72]:
from torch_geometric.transforms import VirtualNode
import copy

qm9_vn = copy.deepcopy(qm9)

transform = VirtualNode()
qm9_vn.transform = transform


In [73]:
# datasets into DataLoader
train_loader = DataLoader(qm9_vn[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_vn[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_vn[test_index:val_index], batch_size=64, shuffle=True)

In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_vn[0].x.shape[1]
dim_h = 64
edge_attr = qm9_vn[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_vn[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

RuntimeError: The size of tensor a (10) must match the size of tensor b (20) at non-singleton dimension 0

In [ ]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_vn.pt"
)

## Centrality

### Betweenness

In [ ]:
import torch
from torch_geometric.utils import to_networkx
import networkx as nx

def pyg_to_networkx_with_attributes(data):
    # Convert PyG data to NetworkX graph, retaining node attributes
    G = to_networkx(data, node_attrs=['x', 'pos', 'z'], edge_attrs=['edge_attr'])

    # Add global attributes manually (if any exist)
    G.graph['y'] = data.y
    G.graph['smiles'] = data.smiles
    G.graph['name'] = data.name
    G.graph['idx'] = data.idx

    return G


In [ ]:
def add_betweenness_centrality(data):
    # Convert PyG data to NetworkX graph, preserving attributes
    G = pyg_to_networkx_with_attributes(data)

    # Compute node betweenness centrality
    node_betweenness = nx.betweenness_centrality(G, normalized=True)
    node_centrality_values = list(node_betweenness.values())
    node_centrality_tensor = torch.tensor(node_centrality_values, dtype=torch.float).view(-1, 1)

    # Append node betweenness centrality to node features
    data.x = torch.cat([data.x, node_centrality_tensor], dim=-1)

    # Compute edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G, normalized=True)
    edge_centrality_values = [edge_betweenness[(u, v)] for u, v in G.edges()]
    edge_centrality_tensor = torch.tensor(edge_centrality_values, dtype=torch.float).view(-1, 1)

    # Append edge betweenness centrality to edge features
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data

In [ ]:
import copy
qm9_data = qm9[:data_size]
qm9_centrality = copy.deepcopy(qm9_data)
qm9_bet = []
for data in qm9_centrality:
    data_bet = add_betweenness_centrality(data)
    qm9_bet.append(data_bet)
    

In [ ]:
# datasets into DataLoader
train_loader = DataLoader(qm9_bet[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_bet[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_bet[test_index:val_index], batch_size=64, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_bet[0].x.shape[1]
dim_h = 64
edge_attr = qm9_bet[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_bet[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [ ]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_bet.pt"
)

### Degree

In [ ]:
def add_centrality_to_node_features(data, centrality_measure='degree'):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Compute the centrality measure
    if centrality_measure == 'degree':
        centrality = nx.degree_centrality(G)
    elif centrality_measure == 'closeness':
        centrality = nx.closeness_centrality(G)
    elif centrality_measure == 'eigenvector':
        if not nx.is_connected(G):
        # Handle connected components separately
            centrality = {}
            for component in nx.connected_components(G):
                subgraph = G.subgraph(component)
                sub_centrality = nx.eigenvector_centrality(subgraph, max_iter=500, tol=1e-4)
                centrality.update(sub_centrality)
        else:
            centrality = nx.eigenvector_centrality(G, max_iter=500, tol=1e-4)
    else:
        raise ValueError(f'Unknown centrality measure: {centrality_measure}')

    # Convert centrality to tensor and add as node feature
    centrality_values = list(centrality.values())
    centrality_tensor = (centrality_tensor - centrality_tensor.mean()) / (centrality_tensor.std() + 1e-8)
    data.x = torch.cat([data.x, centrality_tensor], dim=-1)

    return data

In [ ]:
import copy
qm9_deg = []
for data in qm9_centrality:
    data_deg = add_centrality_to_node_features(data, centrality_measure='degree')
    qm9_deg.append(data_deg)
    

In [ ]:
# datasets into DataLoader
train_loader = DataLoader(qm9_deg[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_deg[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_deg[test_index:val_index], batch_size=64, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_deg[0].x.shape[1]
dim_h = 64
edge_attr = qm9_deg[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_deg[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [ ]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_deg.pt"
)

In [ ]:
import copy
qm9_clo = []
for data in qm9_centrality:
    data_clo = add_centrality_to_node_features(data, centrality_measure='closeness')
    qm9_clo.append(data_deg)
    

In [ ]:
# datasets into DataLoader
train_loader = DataLoader(qm9_clo[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_clo[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_clo[test_index:val_index], batch_size=64, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_clo[0].x.shape[1]
dim_h = 64
edge_attr = qm9_clo[0].edge_attr.shape[1]
from torch_geometric.utils import degree
# Compute in-degree histogram over training data.
deg = torch.zeros(10, dtype=torch.long)
for data in qm9_clo[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [ ]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_clo.pt"
)

# Target = 1

# Refresh

In [77]:
import os
import math
import copy

import numpy as np
import pickle
import networkx as nx
import pandas as pd

import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import QM9
import matplotlib.pyplot as plt
from torch_geometric.utils import from_networkx, to_networkx, to_undirected, to_scipy_sparse_matrix
from torch_geometric.data.data import Data
from torch_geometric.loader import DataLoader

In [78]:
qm9 = QM9(root='dataset/QM9')

In [79]:
y_target = pd.DataFrame(qm9.data.y.numpy())

qm9 = qm9.shuffle()

In [80]:
# data split
data_size = 30000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)

In [81]:
qm9_dataset = qm9[:data_size]

In [82]:
qm9_dataset[0]

Data(x=[21, 11], edge_index=[2, 44], edge_attr=[44, 4], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

# Transformations

## Virtual Node

In [83]:
from torch_geometric.transforms import VirtualNode
import copy

qm9_vn = copy.deepcopy(qm9_dataset)

transform = VirtualNode()
qm9_vn.transform = transform


In [84]:
qm9_vn[0]

Data(x=[22, 11], edge_index=[2, 86], edge_attr=[86, 4], y=[1, 19], pos=[22, 3], z=[22], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1], edge_type=[86])

## Centrality

### Betweenness

In [85]:
import torch
from torch_geometric.utils import to_networkx
import networkx as nx

def pyg_to_networkx_with_attributes(data):
    # Convert PyG data to NetworkX graph, retaining node attributes
    G = to_networkx(data, node_attrs=['x', 'pos', 'z'], edge_attrs=['edge_attr'])

    # Add global attributes manually (if any exist)
    G.graph['y'] = data.y
    G.graph['smiles'] = data.smiles
    G.graph['name'] = data.name
    G.graph['idx'] = data.idx

    return G


In [86]:
def add_betweenness_centrality(data):
    # Convert PyG data to NetworkX graph, preserving attributes
    G = pyg_to_networkx_with_attributes(data)

    # Compute node betweenness centrality
    node_betweenness = nx.betweenness_centrality(G, normalized=True)
    node_centrality_values = list(node_betweenness.values())
    node_centrality_tensor = torch.tensor(node_centrality_values, dtype=torch.float).view(-1, 1)

    # Append node betweenness centrality to node features
    data.x = torch.cat([data.x, node_centrality_tensor], dim=-1)

    # Compute edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G, normalized=True)
    edge_centrality_values = [edge_betweenness[(u, v)] for u, v in G.edges()]
    edge_centrality_tensor = torch.tensor(edge_centrality_values, dtype=torch.float).view(-1, 1)

    # Append edge betweenness centrality to edge features
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data

In [87]:
import copy
qm9_centrality = copy.deepcopy(qm9_dataset)
qm9_bet = []
for data in qm9_centrality:
    data_bet = add_betweenness_centrality(data)
    qm9_bet.append(data_bet)
    

In [88]:
qm9_bet[0]

Data(x=[21, 12], edge_index=[2, 44], edge_attr=[44, 5], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

### Degree, Closeness, Eigenvector Centrality

In [149]:
def add_centrality_to_node_features(data, centrality_measure='degree'):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Compute the centrality measure
    if centrality_measure == 'degree':
        centrality = nx.degree_centrality(G)
    elif centrality_measure == 'closeness':
        centrality = nx.closeness_centrality(G)
    elif centrality_measure == 'eigenvector':
        if G.is_directed():
            G = G.to_undirected()
        if not nx.is_connected(G):
        # Handle connected components separately
            centrality = {}
            for component in nx.connected_components(G):
                subgraph = G.subgraph(component)
                sub_centrality = nx.eigenvector_centrality(subgraph, max_iter=500, tol=1e-4)
                centrality.update(sub_centrality)
        else:
            centrality = nx.eigenvector_centrality(G, max_iter=500, tol=1e-4)
    else:
        raise ValueError(f'Unknown centrality measure: {centrality_measure}')

    # Convert centrality to tensor and add as node feature
    centrality_values = list(centrality.values())
    centrality_tensor = torch.tensor(centrality_values, dtype=torch.float).view(-1, 1)
    centrality_tensor = (centrality_tensor - centrality_tensor.mean()) / (centrality_tensor.std() + 1e-8)
    data.x = torch.cat([data.x, centrality_tensor], dim=-1)


    return data

In [90]:
qm9_deg = []
for data in qm9_centrality:
    data_deg = add_centrality_to_node_features(data, centrality_measure='degree')
    qm9_deg.append(data_deg)
    

In [91]:
qm9_deg[0]

Data(x=[21, 12], edge_index=[2, 44], edge_attr=[44, 4], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

In [152]:
qm9_clo = []
for data in qm9_centrality:
    data_clo = add_centrality_to_node_features(data, centrality_measure='closeness')
    qm9_clo.append(data_clo)
    

In [93]:
qm9_clo[0]

Data(x=[21, 12], edge_index=[2, 44], edge_attr=[44, 4], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

In [96]:
qm9_eig = []
for data in qm9_centrality:
    data_eig = add_centrality_to_node_features(data, centrality_measure='eigenvector')
    qm9_eig.append(data_eig)
    

In [97]:
qm9_eig[0]

Data(x=[21, 12], edge_index=[2, 44], edge_attr=[44, 4], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

## Subgraph Extraction

In [98]:
def extract_local_subgraph_features_with_edges(data, radius=2):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Initialize lists to store node and edge features
    subgraph_sizes = []
    subgraph_degrees = []
    edge_subgraph_centralities = []

    # Compute node-level subgraph features
    for node in G.nodes():
        subgraph = nx.ego_graph(G, node, radius=radius)
        subgraph_size = subgraph.number_of_nodes()
        subgraph_degree = np.mean([d for n, d in subgraph.degree()])

        subgraph_sizes.append(subgraph_size)
        subgraph_degrees.append(subgraph_degree)

    # Convert node features to tensors
    subgraph_sizes_tensor = torch.tensor(subgraph_sizes, dtype=torch.float).view(-1, 1)
    subgraph_degrees_tensor = torch.tensor(subgraph_degrees, dtype=torch.float).view(-1, 1)
    data.x = torch.cat([data.x, subgraph_sizes_tensor, subgraph_degrees_tensor], dim=-1)

    # Compute edge-level subgraph features
    for u, v in G.edges():
        subgraph_u = nx.ego_graph(G, u, radius=radius)
        edge_centrality = nx.edge_betweenness_centrality(subgraph_u, normalized=True).get((u, v), 0)
        edge_subgraph_centralities.append(edge_centrality)

    # Convert edge features to tensor and concatenate to edge_attr
    edge_centrality_tensor = torch.tensor(edge_subgraph_centralities, dtype=torch.float).view(-1, 1)
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data


In [99]:
qm9_sub = copy.deepcopy(qm9_dataset)
qm9_SE = []
for data in qm9_sub:
    data_sub = extract_local_subgraph_features_with_edges(data, radius=2)
    qm9_SE.append(data_sub)
    

In [100]:
qm9_SE[0]

Data(x=[21, 13], edge_index=[2, 44], edge_attr=[44, 5], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

## Graph Encoding

In [101]:
from torch_geometric.transforms import AddLaplacianEigenvectorPE

qm9_GE = copy.deepcopy(qm9_dataset)

transform = AddLaplacianEigenvectorPE(k=2, attr_name = None)
qm9_GE.transform = transform

In [102]:
qm9_GE[0]

Data(x=[21, 13], edge_index=[2, 44], edge_attr=[44, 4], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

## Extra Node

In [107]:
import torch
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

def add_extra_node_on_each_edge(data):
    # Convert PyG data to a NetworkX graph for easier manipulation
    G = to_networkx(data, node_attrs=['x'], edge_attrs = ['edge_attr'])
    
    # Original number of nodes
    num_original_nodes = G.number_of_nodes()
    
    # Prepare lists for new features
    edges = list(G.edges(data=True))
    new_node_features = []
    new_edges_src = []
    new_edges_dst = []
    new_edge_features = []

    for u, v, edge_data in edges:
        # Remove the original edge
        G.remove_edge(u, v)

        # Create new node as the mean of connected node features
        new_node_id = num_original_nodes + len(new_node_features)
        new_node_feature = (data.x[u] + data.x[v]) / 2
        new_node_features.append(new_node_feature)
        
        # Add new node with feature
        G.add_node(new_node_id, x=new_node_feature)

        # Add edges from new node to each original node
        G.add_edge(u, new_node_id)
        G.add_edge(new_node_id, v)

        # Use original edge feature for each new edge
        edge_feature = edge_data['edge_attr']
        edge_feature_tensor = (
            edge_feature if isinstance(edge_feature, torch.Tensor) else torch.tensor(edge_feature)
        )
        new_edge_features.append(edge_feature_tensor)  # for edge (u, new_node_id)
        new_edge_features.append(edge_feature_tensor)  # for edge (new_node_id, v)
    
    # Convert back to PyG Data object
    modified_data = from_networkx(G)

    # Update node features
    modified_data.x = torch.cat([data.x, torch.stack(new_node_features)], dim=0)

    # Update edge features to include only the new edges
    modified_data.edge_attr = torch.stack(new_edge_features)  # Only include new edge features

    # Preserve any additional global attributes
    modified_data.y = data.y
    modified_data.smiles = data.smiles
    modified_data.name = data.name
    modified_data.idx = data.idx
    modified_data.pos = data.pos
    modified_data.z = data.z
    
    return modified_data

In [108]:
qm9_ExN = []
ExN_set = copy.deepcopy(qm9_dataset)
for data in ExN_set:  # Process 100 molecules as an example
    modified_data = add_extra_node_on_each_edge(data)
    qm9_ExN.append(modified_data)

In [109]:
qm9_ExN[0]

Data(x=[65, 11], edge_index=[2, 88], edge_attr=[88, 4], y=[1, 19], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1], pos=[21, 3], z=[21])

## Distance Encoding

# Model

## PNA

In [116]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import PNAConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class PNANet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr, aggregators, scalers, deg):
        super(PNANet, self).__init__()
        
        # Define PNA layers with specified aggregators, scalers, and degree tensor
        self.conv1 = PNAConv(
            in_channels=num_node_features,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv2 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv3 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        
        # Define linear layers for final graph-level output
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Pass node features and edge attributes through PNA layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

## GIN

In [157]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINEConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class GINENet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr):
        super(GINENet, self).__init__()
        
        # Define GINE layers with the specified edge_dim
        self.conv1 = GINEConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        
        # Define linear layers for classification or regression
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Pass node features and edge attributes through GINE layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

# Processing

In [117]:
def training(loader, model, loss, optimizer):
    """Training one epoch

    Args:
        loader (DataLoader): loader (DataLoader): training data divided into batches
        model (nn.Module): GNN model to train on
        loss (nn.functional): loss function to use during training
        optimizer (torch.optim): optimizer during training

    Returns:
        float: training loss
    """
    model.train()

    current_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        data.x = data.x.float()

        out = model(data)

        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        current_loss += l / len(loader)
        l.backward()
        optimizer.step()
    return current_loss, model

def validation(loader, model, loss):
    """Validation

    Args:
        loader (DataLoader): validation set in batches
        model (nn.Module): current trained model
        loss (nn.functional): loss function

    Returns:
        float: validation loss
    """
    model.eval()
    val_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        val_loss += l / len(loader)
    return val_loss

@torch.no_grad()
def testing(loader, model):
    """Testing

    Args:
        loader (DataLoader): test dataset
        model (nn.Module): trained model

    Returns:
        float: test loss
    """
    loss = torch.nn.MSELoss()
    test_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        # NOTE
        # out = out.view(d.y.size())
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        test_loss += l / len(loader)


    return test_loss

In [118]:
def train_epochs(epochs, model, train_loader, val_loader, path):
    """Training over all epochs

    Args:
        epochs (int): number of epochs to train for
        model (nn.Module): the current model
        train_loader (DataLoader): training data in batches
        val_loader (DataLoader): validation data in batches
        path (string): path to save the best model

    Returns:
        array: returning train and validation losses over all epochs, prediction and ground truth values for training data in the last epoch
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    loss = torch.nn.L1Loss()

    train_loss = np.empty(epochs)
    val_loss = np.empty(epochs)
    best_loss = math.inf

    for epoch in range(1, epochs):
        epoch_loss, model = training(train_loader, model, loss, optimizer)
        v_loss = validation(val_loader, model, loss)
        if v_loss < best_loss:
            best_loss = v_loss
            torch.save(model.state_dict(), path)
        for data in train_loader:
            data = data.to(device)
            out = model(data)

        train_loss[epoch] = epoch_loss.detach().cpu().numpy()
        val_loss[epoch] = v_loss.detach().cpu().numpy()

        # print current train and val loss
        if epoch % 5 == 0:
            print(
                "Epoch: "
                + str(epoch)
                + ", Train loss: "
                + str(epoch_loss.item())
                + ", Val loss: "
                + str(v_loss.item())
            )
    return best_loss, train_loss, val_loss

## Target 0

In [119]:
target = 0
qm9_dataset.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_dataset.data.y[0:train_index].mean()
data_std = qm9_dataset.data.y[0:train_index].std()

qm9_dataset.data.y = (qm9_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_dataset[test_index:val_index], batch_size=64, shuffle=True)

/home/lizhifei/.local/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [120]:
qm9_dataset[0]

Data(x=[21, 11], edge_index=[2, 44], edge_attr=[44, 4], y=[1], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

In [121]:
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


In [122]:
aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [123]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [124]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model.pt"
)

Epoch: 5, Train loss: 0.5401504635810852, Val loss: 0.5000472068786621
Epoch: 10, Train loss: 0.5032700300216675, Val loss: 0.48793160915374756
Epoch: 15, Train loss: 0.49474871158599854, Val loss: 0.46651947498321533
Epoch: 20, Train loss: 0.4784812927246094, Val loss: 0.448507159948349
Epoch: 25, Train loss: 0.4720373749732971, Val loss: 0.44866734743118286
Epoch: 30, Train loss: 0.4631823003292084, Val loss: 0.44419562816619873
Epoch: 35, Train loss: 0.4609650671482086, Val loss: 0.448944628238678
Epoch: 40, Train loss: 0.4551342725753784, Val loss: 0.45710673928260803
Epoch: 45, Train loss: 0.4479655623435974, Val loss: 0.4450874626636505
Epoch: 50, Train loss: 0.4539875090122223, Val loss: 0.4534499943256378
Epoch: 55, Train loss: 0.44478073716163635, Val loss: 0.4447292983531952
Epoch: 60, Train loss: 0.44454485177993774, Val loss: 0.4304903745651245
Epoch: 65, Train loss: 0.44563212990760803, Val loss: 0.43736425042152405
Epoch: 70, Train loss: 0.4467959702014923, Val loss: 0.43

In [125]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/1113277864.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model.pt"))


Test Loss for PNA: 0.3822985887527466


In [126]:
target = 0
qm9_vn.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_vn.data.y[0:train_index].mean()
data_std = qm9_vn.data.y[0:train_index].std()

qm9_vn.data.y = (qm9_vn.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_vn[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_vn[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_vn[test_index:val_index], batch_size=64, shuffle=True)

In [127]:
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_vn[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_vn[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


In [128]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_vn[0].x.shape[1]
dim_h = 64
edge_attr = qm9_vn[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [129]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_vn.pt"
)

Epoch: 5, Train loss: 0.5329497456550598, Val loss: 0.5096242427825928
Epoch: 10, Train loss: 0.5075181722640991, Val loss: 0.5027762651443481
Epoch: 15, Train loss: 0.49214962124824524, Val loss: 0.46368408203125
Epoch: 20, Train loss: 0.47868314385414124, Val loss: 0.4604070782661438
Epoch: 25, Train loss: 0.4696469008922577, Val loss: 0.46889686584472656
Epoch: 30, Train loss: 0.46168792247772217, Val loss: 0.4555462598800659
Epoch: 35, Train loss: 0.4575309455394745, Val loss: 0.46633151173591614
Epoch: 40, Train loss: 0.4554443955421448, Val loss: 0.4480733275413513
Epoch: 45, Train loss: 0.4494090676307678, Val loss: 0.4439085125923157
Epoch: 50, Train loss: 0.44441458582878113, Val loss: 0.43937817215919495
Epoch: 55, Train loss: 0.44128888845443726, Val loss: 0.430616557598114
Epoch: 60, Train loss: 0.44125163555145264, Val loss: 0.42584362626075745
Epoch: 65, Train loss: 0.43428319692611694, Val loss: 0.42056283354759216
Epoch: 70, Train loss: 0.43393269181251526, Val loss: 0.

In [130]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_vn.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/3144575080.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_vn.pt"))


Test Loss for PNA: 0.34801554679870605


In [131]:
from torch_geometric.data import InMemoryDataset

# Define a custom InMemoryDataset to hold the processed QM9 data with betweenness centrality
class CustomQM9Dataset(InMemoryDataset):
    def __init__(self, data_list):
        super(CustomQM9Dataset, self).__init__()
        self.data, self.slices = self.collate(data_list)

# Create an InMemoryDataset from the list of transformed graphs
qm9_bet_dataset = CustomQM9Dataset(qm9_bet)

In [132]:
qm9_bet_dataset[0]

Data(x=[21, 12], edge_index=[2, 44], edge_attr=[44, 5], y=[1, 19], pos=[21, 3], z=[21], smiles='[H]C([H])([H])C([H])([H])[C@]1([H])OC([H])([H])[C@]2([H])O[C@]2([H])C1([H])[H]', name='gdb_115914', idx=[1])

In [140]:
target = 0
qm9_bet_dataset.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_bet_dataset.data.y[0:train_index].mean()
data_std = qm9_bet_dataset.data.y[0:train_index].std()

qm9_bet_dataset.data.y = (qm9_bet_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_bet_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_bet_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_bet_dataset[test_index:val_index], batch_size=64, shuffle=True)

In [141]:
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_bet_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_bet_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


In [143]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_bet_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_bet_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [144]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_bet.pt"
)

Epoch: 5, Train loss: 0.783633828163147, Val loss: 1.0525089502334595


KeyboardInterrupt: 

In [153]:
qm9_clo_dataset = CustomQM9Dataset(qm9_clo)

target = 0
qm9_clo_dataset.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_clo_dataset.data.y[0:train_index].mean()
data_std = qm9_clo_dataset.data.y[0:train_index].std()

qm9_clo_dataset.data.y = (qm9_clo_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_clo_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_clo_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_clo_dataset[test_index:val_index], batch_size=64, shuffle=True)

In [154]:
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_clo_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_clo_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


In [155]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_clo_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_clo_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [156]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "PNA_0_model_clo.pt"
)

Epoch: 5, Train loss: 0.7838258743286133, Val loss: 1.0532243251800537


KeyboardInterrupt: 

# GIN

In [158]:
target = 0
qm9_dataset.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_dataset.data.y[0:train_index].mean()
data_std = qm9_dataset.data.y[0:train_index].std()

qm9_dataset.data.y = (qm9_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_dataset[test_index:val_index], batch_size=64, shuffle=True)

num_features = qm9_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_dataset[0].edge_attr.shape[1]
model = GINENet(num_features, dim_h, edge_attr).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
gin_best_loss, gin_train_loss, gin_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "GIN_0_model.pt"
)

Epoch: 5, Train loss: 0.500133752822876, Val loss: 0.508162260055542
Epoch: 10, Train loss: 0.473743200302124, Val loss: 0.4478645622730255
Epoch: 15, Train loss: 0.45926687121391296, Val loss: 0.4447709321975708
Epoch: 20, Train loss: 0.44492265582084656, Val loss: 0.428652286529541
Epoch: 25, Train loss: 0.43891581892967224, Val loss: 0.43907877802848816
Epoch: 30, Train loss: 0.43219250440597534, Val loss: 0.40929269790649414
Epoch: 35, Train loss: 0.42880168557167053, Val loss: 0.42619621753692627
Epoch: 40, Train loss: 0.42239511013031006, Val loss: 0.41917648911476135
Epoch: 45, Train loss: 0.4182688593864441, Val loss: 0.4052686095237732
Epoch: 50, Train loss: 0.41259950399398804, Val loss: 0.39702659845352173
Epoch: 55, Train loss: 0.4112132489681244, Val loss: 0.3918880522251129
Epoch: 60, Train loss: 0.40784984827041626, Val loss: 0.4035443365573883
Epoch: 65, Train loss: 0.4021839201450348, Val loss: 0.41469377279281616
Epoch: 70, Train loss: 0.4013875722885132, Val loss: 0.

In [159]:
# load our model
model = GINENet(num_features, dim_h, edge_attr).to(device)
model.load_state_dict(torch.load("GIN_0_model.pt"))

# calculate test loss
gin_test_loss = testing(test_loader, model)

print("Test Loss for GIN: " + str(gin_test_loss.item()))


/tmp/ipykernel_234347/1724566235.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("GIN_0_model.pt"))


Test Loss for GIN: 0.29048705101013184


In [160]:
target = 0
qm9_clo_dataset.data.y = torch.Tensor(y_target[target])

# normalizing the data
data_mean = qm9_clo_dataset.data.y[0:train_index].mean()
data_std = qm9_clo_dataset.data.y[0:train_index].std()

qm9_clo_dataset.data.y = (qm9_clo_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_clo_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_clo_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_clo_dataset[test_index:val_index], batch_size=64, shuffle=True)

num_features = qm9_clo_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_clo_dataset[0].edge_attr.shape[1]
model = GINENet(num_features, dim_h, edge_attr).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
gin_best_loss, gin_train_loss, gin_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "GIN_0_model_clo.pt"
)

Epoch: 5, Train loss: 0.7835645079612732, Val loss: 1.0414453744888306


KeyboardInterrupt: 

# Fix Test

In [163]:
qm9 = QM9(root='dataset/QM9')

In [164]:
y_target = pd.DataFrame(qm9.data.y.numpy())
qm9.data.y = torch.Tensor(y_target[0])

qm9 = qm9.shuffle()

In [165]:
# data split
data_size = 30000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9.data.y[0:train_index].mean()
data_std = qm9.data.y[0:train_index].std()

qm9.data.y = (qm9.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9[test_index:val_index], batch_size=64, shuffle=True)

In [161]:
import torch
from torch_geometric.utils import to_networkx
import networkx as nx

def pyg_to_networkx_with_attributes(data):
    # Convert PyG data to NetworkX graph, retaining node attributes
    G = to_networkx(data, node_attrs=['x', 'pos', 'z'], edge_attrs=['edge_attr'])

    # Add global attributes manually (if any exist)
    G.graph['y'] = data.y
    G.graph['smiles'] = data.smiles
    G.graph['name'] = data.name
    G.graph['idx'] = data.idx

    return G

In [162]:
def add_betweenness_centrality(data):
    # Convert PyG data to NetworkX graph, preserving attributes
    G = pyg_to_networkx_with_attributes(data)

    # Compute node betweenness centrality
    node_betweenness = nx.betweenness_centrality(G, normalized=True)
    node_centrality_values = list(node_betweenness.values())
    node_centrality_tensor = torch.tensor(node_centrality_values, dtype=torch.float).view(-1, 1)

    # Append node betweenness centrality to node features
    data.x = torch.cat([data.x, node_centrality_tensor], dim=-1)

    # Compute edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G, normalized=True)
    edge_centrality_values = [edge_betweenness[(u, v)] for u, v in G.edges()]
    edge_centrality_tensor = torch.tensor(edge_centrality_values, dtype=torch.float).view(-1, 1)

    # Append edge betweenness centrality to edge features
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data

In [166]:
import copy
qm9_data = qm9[:data_size]
qm9_centrality = copy.deepcopy(qm9_data)
qm9_bet = []
for data in qm9_centrality:
    data_bet = add_betweenness_centrality(data)
    qm9_bet.append(data_bet)

In [167]:
# datasets into DataLoader
train_loader = DataLoader(qm9_bet[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_bet[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_bet[test_index:val_index], batch_size=64, shuffle=True)

In [170]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_bet[0].x.shape[1]
dim_h = 64
edge_attr = qm9_bet[0].edge_attr.shape[1]
model = GINENet(num_features, dim_h, edge_attr).to(device)

In [172]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
gin_best_loss, gin_train_loss, gin_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "GIN_0_model_bet.pt"
)

Epoch: 5, Train loss: 0.4425533711910248, Val loss: 0.44334885478019714
Epoch: 10, Train loss: 0.4316164553165436, Val loss: 0.43632057309150696
Epoch: 15, Train loss: 0.4227544665336609, Val loss: 0.42684265971183777
Epoch: 20, Train loss: 0.41685619950294495, Val loss: 0.40088707208633423
Epoch: 25, Train loss: 0.4120086133480072, Val loss: 0.4001424312591553
Epoch: 30, Train loss: 0.4075751006603241, Val loss: 0.3974928855895996
Epoch: 35, Train loss: 0.4040742516517639, Val loss: 0.3979135751724243
Epoch: 40, Train loss: 0.3998916447162628, Val loss: 0.4012698233127594
Epoch: 45, Train loss: 0.398292601108551, Val loss: 0.39024898409843445
Epoch: 50, Train loss: 0.39782747626304626, Val loss: 0.3999740779399872
Epoch: 55, Train loss: 0.39592859148979187, Val loss: 0.38609248399734497
Epoch: 60, Train loss: 0.38877934217453003, Val loss: 0.39995473623275757
Epoch: 65, Train loss: 0.39246419072151184, Val loss: 0.38175129890441895
Epoch: 70, Train loss: 0.3892141282558441, Val loss: 

In [173]:
# load our model
model = GINENet(num_features, dim_h, edge_attr).to(device)
model.load_state_dict(torch.load("GIN_0_model_bet.pt"))

# calculate test loss
gin_test_loss = testing(test_loader, model)

print("Test Loss for GIN: " + str(gin_test_loss.item()))


/tmp/ipykernel_234347/2409921137.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("GIN_0_model_bet.pt"))


Test Loss for GIN: 0.30340367555618286


In [181]:
from torch_geometric.nn import GINConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool
class GIN(torch.nn.Module):
    """Graph Isomorphism Network class with 3 GINConv layers and 2 linear layers"""

    def __init__(self, num_features, dim_h):
        """Initializing GIN class

        Args:
            dim_h (int): the dimension of hidden layers
        """
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(num_features, dim_h), BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU())
        )
        self.conv2 = GINConv(
            Sequential(
                Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU()
            )
        )
        self.conv3 = GINConv(
            Sequential(
                Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU()
            )
        )
        self.lin1 = Linear(dim_h, dim_h)
        self.lin2 = Linear(dim_h, 1)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        # Node embeddings
        h = self.conv1(x, edge_index)
        h = h.relu()
        h = self.conv2(h, edge_index)
        h = h.relu()
        h = self.conv3(h, edge_index)

        # Graph-level readout
        h = global_add_pool(h, batch)

        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h

In [182]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_bet[0].x.shape[1]
dim_h = 64
edge_attr = qm9_bet[0].edge_attr.shape[1]
model = GIN(num_features, dim_h).to(device)

In [183]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
gin_best_loss, gin_train_loss, gin_val_loss = train_epochs(
    epochs, model, train_loader, test_loader, "GINsimple_0_model.pt"
)

Epoch: 5, Train loss: 0.4953996241092682, Val loss: 0.4647423326969147
Epoch: 10, Train loss: 0.46984025835990906, Val loss: 0.4653249979019165
Epoch: 15, Train loss: 0.4588661789894104, Val loss: 0.4172241687774658
Epoch: 20, Train loss: 0.44611597061157227, Val loss: 0.42408013343811035
Epoch: 25, Train loss: 0.4390184283256531, Val loss: 0.4167090356349945
Epoch: 30, Train loss: 0.4352353513240814, Val loss: 0.3947773277759552
Epoch: 35, Train loss: 0.43027421832084656, Val loss: 0.4067845940589905
Epoch: 40, Train loss: 0.4220568835735321, Val loss: 0.426036536693573
Epoch: 45, Train loss: 0.42082715034484863, Val loss: 0.3889799416065216
Epoch: 50, Train loss: 0.41620951890945435, Val loss: 0.3973839581012726
Epoch: 55, Train loss: 0.4133622646331787, Val loss: 0.3881092071533203
Epoch: 60, Train loss: 0.4112772047519684, Val loss: 0.38772523403167725
Epoch: 65, Train loss: 0.4093972444534302, Val loss: 0.3814649283885956
Epoch: 70, Train loss: 0.4051569998264313, Val loss: 0.3862

In [184]:
# load our model
model = GIN(num_features, dim_h).to(device)
model.load_state_dict(torch.load("GINsimple_0_model.pt"))

# calculate test loss
gin_test_loss = testing(test_loader, model)

print("Test Loss for GIN: " + str(gin_test_loss.item()))


/tmp/ipykernel_234347/671566738.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("GINsimple_0_model.pt"))


Test Loss for GIN: 0.31369003653526306


In [205]:
import os
import math
import copy

import numpy as np
import pickle
import networkx as nx
import pandas as pd

import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import QM9
import matplotlib.pyplot as plt
from torch_geometric.utils import from_networkx, to_networkx, to_undirected, to_scipy_sparse_matrix
from torch_geometric.data.data import Data
from torch_geometric.loader import DataLoader

In [207]:
qm9 = QM9(root='dataset/QM9')
qm9_dataset = qm9[:5000]

y_target = pd.DataFrame(qm9_dataset.data.y.numpy())
qm9_dataset.data.y = torch.Tensor(y_target[0])

qm9_dataset = qm9_dataset.shuffle()

# Transformation

In [208]:
from torch_geometric.transforms import VirtualNode
import copy

qm9_vn = copy.deepcopy(qm9_dataset)

transform = VirtualNode()
qm9_vn.transform = transform


In [221]:
import torch
from torch_geometric.utils import to_networkx
import networkx as nx

def pyg_to_networkx_with_attributes(data):
    # Convert PyG data to NetworkX graph, retaining node attributes
    G = to_networkx(data, node_attrs=['x', 'pos', 'z'], edge_attrs=['edge_attr'])

    # Add global attributes manually (if any exist)
    G.graph['y'] = data.y
    G.graph['smiles'] = data.smiles
    G.graph['name'] = data.name
    G.graph['idx'] = data.idx

    return G

In [222]:
def add_betweenness_centrality(data):
    # Convert PyG data to NetworkX graph, preserving attributes
    G = pyg_to_networkx_with_attributes(data)

    # Compute node betweenness centrality
    node_betweenness = nx.betweenness_centrality(G, normalized=True)
    node_centrality_values = list(node_betweenness.values())
    node_centrality_tensor = torch.tensor(node_centrality_values, dtype=torch.float).view(-1, 1)

    # Append node betweenness centrality to node features
    data.x = torch.cat([data.x, node_centrality_tensor], dim=-1)

    # Compute edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G, normalized=True)
    edge_centrality_values = [edge_betweenness[(u, v)] for u, v in G.edges()]
    edge_centrality_tensor = torch.tensor(edge_centrality_values, dtype=torch.float).view(-1, 1)

    # Append edge betweenness centrality to edge features
    if 'edge_attr' in data:
        data.edge_attr = torch.cat([data.edge_attr, edge_centrality_tensor], dim=-1)
    else:
        data.edge_attr = edge_centrality_tensor

    return data

In [224]:
import copy
qm9_centrality = copy.deepcopy(qm9_dataset)
qm9_bet = []
for data in qm9_centrality:
    data_bet = add_betweenness_centrality(data)
    qm9_bet.append(data_bet)
    

In [209]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_vn.data.y[0:train_index].mean()
data_std = qm9_vn.data.y[0:train_index].std()

qm9_vn.data.y = (qm9_vn.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_vn[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_vn[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_vn[test_index:val_index], batch_size=64, shuffle=True)

In [210]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINEConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class GINENet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr):
        super(GINENet, self).__init__()
        
        # Define GINE layers with the specified edge_dim
        self.conv1 = GINEConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()),
            edge_dim=edge_attr)
        
        # Define linear layers for classification or regression
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Pass node features and edge attributes through GINE layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

In [211]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import PNAConv, global_add_pool
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d

class PNANet(torch.nn.Module):
    def __init__(self, num_node_features, dim_h, edge_attr, aggregators, scalers, deg):
        super(PNANet, self).__init__()
        
        # Define PNA layers with specified aggregators, scalers, and degree tensor
        self.conv1 = PNAConv(
            in_channels=num_node_features,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv2 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        self.conv3 = PNAConv(
            in_channels=dim_h,
            out_channels=dim_h,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            edge_dim=edge_attr
        )
        
        # Define linear layers for final graph-level output
        self.lin1 = Linear(dim_h * 3, dim_h * 3)
        self.lin2 = Linear(dim_h * 3, 1)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Pass node features and edge attributes through PNA layers
        h1 = self.conv1(x, edge_index, edge_attr)
        h2 = self.conv2(h1, edge_index, edge_attr)
        h3 = self.conv3(h2, edge_index, edge_attr)

        # Apply global pooling for graph-level output
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate pooled features and pass through final linear layers
        h = torch.cat((h1, h2, h3), dim=1)
        h = self.lin1(h).relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        return h

In [212]:
def training(loader, model, loss, optimizer):
    """Training one epoch

    Args:
        loader (DataLoader): loader (DataLoader): training data divided into batches
        model (nn.Module): GNN model to train on
        loss (nn.functional): loss function to use during training
        optimizer (torch.optim): optimizer during training

    Returns:
        float: training loss
    """
    model.train()

    current_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        data.x = data.x.float()

        out = model(data)

        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        current_loss += l / len(loader)
        l.backward()
        optimizer.step()
    return current_loss, model

def validation(loader, model, loss):
    """Validation

    Args:
        loader (DataLoader): validation set in batches
        model (nn.Module): current trained model
        loss (nn.functional): loss function

    Returns:
        float: validation loss
    """
    model.eval()
    val_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        val_loss += l / len(loader)
    return val_loss

@torch.no_grad()
def testing(loader, model):
    """Testing

    Args:
        loader (DataLoader): test dataset
        model (nn.Module): trained model

    Returns:
        float: test loss
    """
    loss = torch.nn.MSELoss()
    test_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        # NOTE
        # out = out.view(d.y.size())
        l = loss(out, torch.reshape(data.y, (len(data.y), 1)))
        test_loss += l / len(loader)


    return test_loss

In [216]:
def train_epochs(epochs, model, train_loader, val_loader, path):
    """Training over all epochs

    Args:
        epochs (int): number of epochs to train for
        model (nn.Module): the current model
        train_loader (DataLoader): training data in batches
        val_loader (DataLoader): validation data in batches
        path (string): path to save the best model

    Returns:
        array: returning train and validation losses over all epochs, prediction and ground truth values for training data in the last epoch
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    loss = torch.nn.L1Loss()

    train_loss = np.empty(epochs)
    val_loss = np.empty(epochs)
    best_loss = math.inf

    for epoch in range(1, epochs):
        epoch_loss, model = training(train_loader, model, loss, optimizer)
        v_loss = validation(val_loader, model, loss)
        if v_loss < best_loss:
            best_loss = v_loss
            torch.save(model.state_dict(), path)
        for data in train_loader:
            data = data.to(device)
            out = model(data)

        train_loss[epoch] = epoch_loss.detach().cpu().numpy()
        val_loss[epoch] = v_loss.detach().cpu().numpy()

        # print current train and val loss
        if epoch % 2 == 0:
            print(
                "Epoch: "
                + str(epoch)
                + ", Train loss: "
                + str(epoch_loss.item())
                + ", Val loss: "
                + str(v_loss.item())
            )
    return best_loss, train_loss, val_loss

In [217]:
from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_vn[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_vn[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [218]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_vn[0].x.shape[1]
dim_h = 64
edge_attr = qm9_vn[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

In [219]:
epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_vn.pt"
)

Epoch: 2, Train loss: 0.7384793758392334, Val loss: 0.6605138182640076
Epoch: 4, Train loss: 0.6063035130500793, Val loss: 0.5809478163719177
Epoch: 6, Train loss: 0.5589114427566528, Val loss: 0.5237354040145874
Epoch: 8, Train loss: 0.5486401319503784, Val loss: 0.5280663371086121
Epoch: 10, Train loss: 0.549461841583252, Val loss: 0.5279342532157898
Epoch: 12, Train loss: 0.5425981283187866, Val loss: 0.5196453332901001
Epoch: 14, Train loss: 0.5179474949836731, Val loss: 0.5465250015258789
Epoch: 16, Train loss: 0.5011120438575745, Val loss: 0.47835907340049744
Epoch: 18, Train loss: 0.5008600950241089, Val loss: 0.4659021198749542
Epoch: 20, Train loss: 0.5041476488113403, Val loss: 0.48254144191741943
Epoch: 22, Train loss: 0.4931771755218506, Val loss: 0.45759081840515137
Epoch: 24, Train loss: 0.4926849901676178, Val loss: 0.4597529172897339
Epoch: 26, Train loss: 0.4816162884235382, Val loss: 0.4646759033203125
Epoch: 28, Train loss: 0.4744998812675476, Val loss: 0.45399782061

In [220]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_vn.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/3144575080.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_vn.pt"))


Test Loss for PNA: 0.4123680591583252


## Bet

In [240]:
class CustomQM9Dataset(InMemoryDataset):
    def __init__(self, data_list):
        super(CustomQM9Dataset, self).__init__()
        self.data, self.slices = self.collate(data_list)
        
qm9_bet_dataset = CustomQM9Dataset(qm9_bet)

In [242]:
qm9_bet_dataset[0]

Data(x=[16, 12], edge_index=[2, 32], edge_attr=[32, 5], y=[1], pos=[16, 3], z=[16], smiles='[H]OC1=C(N([H])[H])C([H])=C(C([H])([H])[H])N1[H]', name='gdb_4492', idx=[1])

In [244]:
qm9_bet[0]

Data(x=[16, 12], edge_index=[2, 32], edge_attr=[32, 5], y=[1], pos=[16, 3], z=[16], smiles='[H]OC1=C(N([H])[H])C([H])=C(C([H])([H])[H])N1[H]', name='gdb_4492', idx=[1])

In [241]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_bet_dataset.data.y[0:train_index].mean()
data_std = qm9_bet_dataset.data.y[0:train_index].std()

qm9_bet_dataset.data.y = (qm9_bet_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_bet_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_bet_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_bet_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_bet_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_bet_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [245]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_bet[0].x.shape[1]
dim_h = 64
edge_attr = qm9_bet[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_bet.pt"
)

Epoch: 2, Train loss: 0.6587304472923279, Val loss: 0.5765650868415833
Epoch: 4, Train loss: 0.583828866481781, Val loss: 0.5183671712875366
Epoch: 6, Train loss: 0.5375897288322449, Val loss: 0.48116886615753174
Epoch: 8, Train loss: 0.5034983158111572, Val loss: 0.5219138264656067
Epoch: 10, Train loss: 0.5191171169281006, Val loss: 0.45960476994514465
Epoch: 12, Train loss: 0.4951198697090149, Val loss: 0.45158034563064575
Epoch: 14, Train loss: 0.4819163382053375, Val loss: 0.4444285035133362
Epoch: 16, Train loss: 0.49036213755607605, Val loss: 0.44865548610687256
Epoch: 18, Train loss: 0.48305392265319824, Val loss: 0.45511487126350403
Epoch: 20, Train loss: 0.4717722535133362, Val loss: 0.4234985411167145
Epoch: 22, Train loss: 0.46605774760246277, Val loss: 0.4434279799461365
Epoch: 24, Train loss: 0.4499996304512024, Val loss: 0.4987345337867737
Epoch: 26, Train loss: 0.45014825463294983, Val loss: 0.42489251494407654
Epoch: 28, Train loss: 0.44707202911376953, Val loss: 0.425

In [246]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_bet.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/1572355086.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_bet.pt"))


Test Loss for PNA: 0.38897988200187683


In [247]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_dataset.data.y[0:train_index].mean()
data_std = qm9_dataset.data.y[0:train_index].std()

qm9_dataset.data.y = (qm9_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [248]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model.pt"
)

Epoch: 2, Train loss: 0.6711087226867676, Val loss: 0.6063773036003113
Epoch: 4, Train loss: 0.6090232133865356, Val loss: 0.546333909034729
Epoch: 6, Train loss: 0.5833888053894043, Val loss: 0.5302543640136719
Epoch: 8, Train loss: 0.5726338028907776, Val loss: 0.5908280611038208
Epoch: 10, Train loss: 0.5546010732650757, Val loss: 0.5251883268356323
Epoch: 12, Train loss: 0.5343540906906128, Val loss: 0.49096959829330444
Epoch: 14, Train loss: 0.5265180468559265, Val loss: 0.5319472551345825
Epoch: 16, Train loss: 0.5307227969169617, Val loss: 0.5027675032615662
Epoch: 18, Train loss: 0.5154920220375061, Val loss: 0.4776037931442261
Epoch: 20, Train loss: 0.5007166266441345, Val loss: 0.5244244337081909
Epoch: 22, Train loss: 0.5101432204246521, Val loss: 0.5580991506576538
Epoch: 24, Train loss: 0.49939224123954773, Val loss: 0.4896053075790405
Epoch: 26, Train loss: 0.48717933893203735, Val loss: 0.4847613573074341
Epoch: 28, Train loss: 0.48831191658973694, Val loss: 0.4888888299

In [249]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/1113277864.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model.pt"))


Test Loss for PNA: 0.45471668243408203


# Deg

In [250]:
def add_centrality_to_node_features(data, centrality_measure='degree'):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Compute the centrality measure
    if centrality_measure == 'degree':
        centrality = nx.degree_centrality(G)
    elif centrality_measure == 'closeness':
        centrality = nx.closeness_centrality(G)
    elif centrality_measure == 'eigenvector':
        if G.is_directed():
            G = G.to_undirected()
        if not nx.is_connected(G):
        # Handle connected components separately
            centrality = {}
            for component in nx.connected_components(G):
                subgraph = G.subgraph(component)
                sub_centrality = nx.eigenvector_centrality(subgraph, max_iter=500, tol=1e-4)
                centrality.update(sub_centrality)
        else:
            centrality = nx.eigenvector_centrality(G, max_iter=500, tol=1e-4)
    else:
        raise ValueError(f'Unknown centrality measure: {centrality_measure}')

    # Convert centrality to tensor and add as node feature
    centrality_values = list(centrality.values())
    centrality_tensor = torch.tensor(centrality_values, dtype=torch.float).view(-1, 1)
    centrality_tensor = (centrality_tensor - centrality_tensor.mean()) / (centrality_tensor.std() + 1e-8)
    data.x = torch.cat([data.x, centrality_tensor], dim=-1)


    return data

In [251]:
qm9_deg = []
for data in qm9_centrality:
    data_deg = add_centrality_to_node_features(data, centrality_measure='degree')
    qm9_deg.append(data_deg)
    
    

In [252]:
qm9_deg_dataset = CustomQM9Dataset(qm9_deg)

In [255]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_deg_dataset.data.y[0:train_index].mean()
data_std = qm9_deg_dataset.data.y[0:train_index].std()

qm9_deg_dataset.data.y = (qm9_deg_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_deg_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_deg_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_deg_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_deg_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_deg_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [256]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_deg_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_deg_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_deg.pt"
)

Epoch: 2, Train loss: 0.6537501811981201, Val loss: 0.5716278553009033
Epoch: 4, Train loss: 0.5661993026733398, Val loss: 0.5297939777374268
Epoch: 6, Train loss: 0.538526713848114, Val loss: 0.5175751447677612
Epoch: 8, Train loss: 0.5230590105056763, Val loss: 0.4853278398513794
Epoch: 10, Train loss: 0.5258864164352417, Val loss: 0.4737209379673004
Epoch: 12, Train loss: 0.507095992565155, Val loss: 0.4825216233730316
Epoch: 14, Train loss: 0.48469388484954834, Val loss: 0.4710986912250519
Epoch: 16, Train loss: 0.4969146251678467, Val loss: 0.47007662057876587
Epoch: 18, Train loss: 0.486106276512146, Val loss: 0.4392777681350708
Epoch: 20, Train loss: 0.47254419326782227, Val loss: 0.4269437789916992
Epoch: 22, Train loss: 0.47093844413757324, Val loss: 0.4293811321258545
Epoch: 24, Train loss: 0.4430338442325592, Val loss: 0.42953670024871826
Epoch: 26, Train loss: 0.4539099335670471, Val loss: 0.49615585803985596
Epoch: 28, Train loss: 0.46374017000198364, Val loss: 0.422817617

In [257]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_deg.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


/tmp/ipykernel_234347/390849739.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_deg.pt"))


Test Loss for PNA: 0.3684897720813751


In [258]:
qm9_clo = []
for data in qm9_centrality:
    data_clo = add_centrality_to_node_features(data, centrality_measure='closeness')
    qm9_clo.append(data_clo)

In [259]:
qm9_clo_dataset = CustomQM9Dataset(qm9_clo)

In [260]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_clo_dataset.data.y[0:train_index].mean()
data_std = qm9_clo_dataset.data.y[0:train_index].std()

qm9_clo_dataset.data.y = (qm9_clo_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_clo_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_clo_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_clo_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_clo_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_clo_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [261]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_clo_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_clo_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_clo.pt"
)

Epoch: 2, Train loss: 0.6393109560012817, Val loss: 0.5559122562408447
Epoch: 4, Train loss: 0.5645928382873535, Val loss: 0.5347018241882324
Epoch: 6, Train loss: 0.5396029949188232, Val loss: 0.5057032108306885
Epoch: 8, Train loss: 0.5147683620452881, Val loss: 0.46638962626457214
Epoch: 10, Train loss: 0.49338993430137634, Val loss: 0.44360896944999695
Epoch: 12, Train loss: 0.48792511224746704, Val loss: 0.4718412756919861
Epoch: 14, Train loss: 0.47131863236427307, Val loss: 0.4388616979122162
Epoch: 16, Train loss: 0.4629957377910614, Val loss: 0.40439149737358093
Epoch: 18, Train loss: 0.46282726526260376, Val loss: 0.42124560475349426
Epoch: 20, Train loss: 0.4612669348716736, Val loss: 0.42351841926574707
Epoch: 22, Train loss: 0.4445327818393707, Val loss: 0.41041451692581177
Epoch: 24, Train loss: 0.43824294209480286, Val loss: 0.45043596625328064
Epoch: 26, Train loss: 0.4424562454223633, Val loss: 0.3897877335548401
Epoch: 28, Train loss: 0.4349595010280609, Val loss: 0.4

In [267]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_clo.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


Test Loss for PNA: 0.383576899766922


/tmp/ipykernel_234347/1217052971.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_clo.pt"))


In [268]:
qm9_eig = []
for data in qm9_centrality:
    data_eig = add_centrality_to_node_features(data, centrality_measure='eigenvector')
    qm9_eig.append(data_eig)

In [269]:
qm9_eig_dataset = CustomQM9Dataset(qm9_eig)

In [270]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_eig_dataset.data.y[0:train_index].mean()
data_std = qm9_eig_dataset.data.y[0:train_index].std()

qm9_eig_dataset.data.y = (qm9_eig_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_eig_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_eig_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_eig_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_eig_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_eig_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [271]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_eig_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_eig_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_eig.pt"
)

Epoch: 2, Train loss: 0.6574724316596985, Val loss: 0.5758398771286011
Epoch: 4, Train loss: 0.5719799995422363, Val loss: 0.5563930869102478
Epoch: 6, Train loss: 0.532564640045166, Val loss: 0.506310760974884
Epoch: 8, Train loss: 0.5254233479499817, Val loss: 0.5619834065437317
Epoch: 10, Train loss: 0.5194610953330994, Val loss: 0.4729815423488617
Epoch: 12, Train loss: 0.5063085556030273, Val loss: 0.4623737037181854
Epoch: 14, Train loss: 0.4895370304584503, Val loss: 0.44933250546455383
Epoch: 16, Train loss: 0.4817868173122406, Val loss: 0.46185553073883057
Epoch: 18, Train loss: 0.46930715441703796, Val loss: 0.4319245517253876
Epoch: 20, Train loss: 0.4580933153629303, Val loss: 0.43622368574142456
Epoch: 22, Train loss: 0.4605036675930023, Val loss: 0.43310990929603577
Epoch: 24, Train loss: 0.45734116435050964, Val loss: 0.4273054301738739
Epoch: 26, Train loss: 0.46473824977874756, Val loss: 0.4969777464866638
Epoch: 28, Train loss: 0.4536641240119934, Val loss: 0.42418268

In [272]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_eig.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


Test Loss for PNA: 0.36430564522743225


/tmp/ipykernel_234347/2637821783.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_eig.pt"))


In [308]:
def extract_local_subgraph_features(data, radius=2):
    # Convert PyG data to NetworkX graph
    G = pyg_to_networkx_with_attributes(data)

    # Initialize a list to store subgraph features for each node
    subgraph_sizes = []
    subgraph_degrees = []
    
    for node in G.nodes():
        # Extract the ego graph (subgraph) around the node
        subgraph = nx.ego_graph(G, node, radius=radius)
        
        # Example feature 1: Size of the subgraph (number of nodes)
        subgraph_size = subgraph.number_of_nodes()
        subgraph_sizes.append(subgraph_size)
        
        # Example feature 2: Average degree of the subgraph
        subgraph_degree = np.mean([d for n, d in subgraph.degree()])
        subgraph_degrees.append(subgraph_degree)
        
    # Convert the features to tensors and add them as node features
    subgraph_sizes_tensor = torch.tensor(subgraph_sizes, dtype=torch.float).view(-1, 1)
    subgraph_degrees_tensor = torch.tensor(subgraph_degrees, dtype=torch.float).view(-1, 1)
    
    # Concatenate the new features to the existing node features
    data.x = torch.cat([data.x, subgraph_sizes_tensor, subgraph_degrees_tensor], dim=-1)
    
    return data

In [309]:
qm9_sub = copy.deepcopy(qm9_dataset)
qm9_SE = []
for data in qm9_sub:
    data_sub = extract_local_subgraph_features_with_edges(data, radius=4)
    qm9_SE.append(data_sub)
    

In [319]:
def add_higher_order_aggregation(data, radius=2):
    """Adds higher-order aggregation information based on neighborhood statistics."""
    G = to_networkx(data, to_undirected=True, node_attrs=['x'])
    subgraph_means = []
    
    for node in G.nodes():
        # Get the subgraph within the specified radius
        subgraph = nx.ego_graph(G, node, radius=radius)
        
        # Ensure each node feature is a tensor
        subgraph_features = []
        for n in subgraph.nodes():
            feature = G.nodes[n]['x']
            feature_tensor = torch.tensor(feature, dtype=torch.float) if isinstance(feature, list) else feature
            subgraph_features.append(feature_tensor)
        
        # Stack the subgraph features and calculate the mean
        subgraph_features = torch.stack(subgraph_features)
        subgraph_mean = subgraph_features.mean(dim=0)
        subgraph_means.append(subgraph_mean)

    # Concatenate the aggregated features with the original node features
    subgraph_means_tensor = torch.stack(subgraph_means)
    data.x = torch.cat([data.x, subgraph_means_tensor], dim=-1)
    
    return data

# Apply the transformation to the dataset
qm9_higher_order = [add_higher_order_aggregation(data) for data in qm9_sub]


In [320]:
qm9_HO_dataset = CustomQM9Dataset(qm9_higher_order)

In [310]:
qm9_SE_dataset = CustomQM9Dataset(qm9_SE)

In [321]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_HO_dataset.data.y[0:train_index].mean()
data_std = qm9_HO_dataset.data.y[0:train_index].std()

qm9_HO_dataset.data.y = (qm9_HO_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_HO_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_HO_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_HO_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_HO_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_HO_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [322]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_HO_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_HO_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_HO.pt"
)

Epoch: 2, Train loss: 0.80887770652771, Val loss: 0.8398736715316772
Epoch: 4, Train loss: 0.8001496195793152, Val loss: 0.8321776390075684
Epoch: 6, Train loss: 0.7987918853759766, Val loss: 0.8343009352684021
Epoch: 8, Train loss: 0.7973417639732361, Val loss: 0.8334730863571167
Epoch: 10, Train loss: 0.7962929606437683, Val loss: 0.8378956317901611
Epoch: 12, Train loss: 0.7957975268363953, Val loss: 0.8349051475524902
Epoch: 14, Train loss: 0.7964848279953003, Val loss: 0.8340446949005127
Epoch: 16, Train loss: 0.795705258846283, Val loss: 0.834448516368866
Epoch: 18, Train loss: 0.7968242168426514, Val loss: 0.8355870246887207
Epoch: 20, Train loss: 0.7952485084533691, Val loss: 0.8312394618988037
Epoch: 22, Train loss: 0.7959997057914734, Val loss: 0.8321973085403442
Epoch: 24, Train loss: 0.794975221157074, Val loss: 0.8321433663368225
Epoch: 26, Train loss: 0.7941954731941223, Val loss: 0.8289918303489685
Epoch: 28, Train loss: 0.7955598831176758, Val loss: 0.8308260440826416
E

KeyboardInterrupt: 

In [317]:
model = GINENet(num_features, dim_h, edge_attr).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
gin_best_loss, gin_train_loss, gin_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "GIN_0_model_SE.pt"
)

Epoch: 2, Train loss: 0.8013623356819153, Val loss: 0.8386300802230835
Epoch: 4, Train loss: 0.7994137406349182, Val loss: 0.8310486674308777
Epoch: 6, Train loss: 0.7961886525154114, Val loss: 0.8380690813064575
Epoch: 8, Train loss: 0.7950918674468994, Val loss: 0.834450364112854
Epoch: 10, Train loss: 0.7949660420417786, Val loss: 0.8336603045463562
Epoch: 12, Train loss: 0.7950272560119629, Val loss: 0.8317904472351074
Epoch: 14, Train loss: 0.796575129032135, Val loss: 0.8325912356376648
Epoch: 16, Train loss: 0.7921549081802368, Val loss: 0.8310779333114624
Epoch: 18, Train loss: 0.7916917204856873, Val loss: 0.8376547694206238
Epoch: 20, Train loss: 0.7950850129127502, Val loss: 0.8332239985466003
Epoch: 22, Train loss: 0.7932614684104919, Val loss: 0.8298947811126709
Epoch: 24, Train loss: 0.7915077805519104, Val loss: 0.8310540914535522
Epoch: 26, Train loss: 0.7924097180366516, Val loss: 0.8337491750717163
Epoch: 28, Train loss: 0.792404294013977, Val loss: 0.8303728103637695

In [301]:
# load our model
model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)
model.load_state_dict(torch.load("PNA_0_model_SE.pt"))

# calculate test loss
pna_test_loss = testing(test_loader, model)

print("Test Loss for PNA: " + str(pna_test_loss.item()))


Test Loss for PNA: 1.055074691772461


/tmp/ipykernel_234347/3159440653.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("PNA_0_model_SE.pt"))


In [323]:
import torch
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

def add_extra_node_on_each_edge(data):
    # Convert PyG data to a NetworkX graph for easier manipulation
    G = to_networkx(data, node_attrs=['x'], edge_attrs = ['edge_attr'])
    
    # Original number of nodes
    num_original_nodes = G.number_of_nodes()
    
    # Prepare lists for new features
    edges = list(G.edges(data=True))
    new_node_features = []
    new_edges_src = []
    new_edges_dst = []
    new_edge_features = []

    for u, v, edge_data in edges:
        # Remove the original edge
        G.remove_edge(u, v)

        # Create new node as the mean of connected node features
        new_node_id = num_original_nodes + len(new_node_features)
        new_node_feature = (data.x[u] + data.x[v]) / 2
        new_node_features.append(new_node_feature)
        
        # Add new node with feature
        G.add_node(new_node_id, x=new_node_feature)

        # Add edges from new node to each original node
        G.add_edge(u, new_node_id)
        G.add_edge(new_node_id, v)

        # Use original edge feature for each new edge
        edge_feature = edge_data['edge_attr']
        edge_feature_tensor = (
            edge_feature if isinstance(edge_feature, torch.Tensor) else torch.tensor(edge_feature)
        )
        new_edge_features.append(edge_feature_tensor)  # for edge (u, new_node_id)
        new_edge_features.append(edge_feature_tensor)  # for edge (new_node_id, v)
    
    # Convert back to PyG Data object
    modified_data = from_networkx(G)

    # Update node features
    modified_data.x = torch.cat([data.x, torch.stack(new_node_features)], dim=0)

    # Update edge features to include only the new edges
    modified_data.edge_attr = torch.stack(new_edge_features)  # Only include new edge features

    # Preserve any additional global attributes
    modified_data.y = data.y
    modified_data.smiles = data.smiles
    modified_data.name = data.name
    modified_data.idx = data.idx
    modified_data.pos = data.pos
    modified_data.z = data.z
    
    return modified_data

In [324]:
qm9_ExN = []
ExN_set = copy.deepcopy(qm9_dataset)
for data in ExN_set:  # Process 100 molecules as an example
    modified_data = add_extra_node_on_each_edge(data)
    qm9_ExN.append(modified_data)

In [325]:
qm9_exN_dataset = CustomQM9Dataset(qm9_ExN)

In [327]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_exN_dataset.data.y[0:train_index].mean()
data_std = qm9_exN_dataset.data.y[0:train_index].std()

qm9_exN_dataset.data.y = (qm9_exN_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_exN_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_exN_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_exN_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_exN_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_exN_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [328]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_exN_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_exN_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_exN.pt"
)

Epoch: 2, Train loss: 0.803960382938385, Val loss: 0.8310350179672241
Epoch: 4, Train loss: 0.7991498112678528, Val loss: 0.8361106514930725
Epoch: 6, Train loss: 0.7971280813217163, Val loss: 0.8348326683044434
Epoch: 8, Train loss: 0.7993593215942383, Val loss: 0.8351888656616211
Epoch: 10, Train loss: 0.7971411347389221, Val loss: 0.835618793964386
Epoch: 12, Train loss: 0.7956464290618896, Val loss: 0.833645761013031
Epoch: 14, Train loss: 0.7958497405052185, Val loss: 0.8305310010910034
Epoch: 16, Train loss: 0.7952824234962463, Val loss: 0.8333723545074463
Epoch: 18, Train loss: 0.7941283583641052, Val loss: 0.8328083157539368
Epoch: 20, Train loss: 0.7943302989006042, Val loss: 0.8299891352653503
Epoch: 22, Train loss: 0.795461118221283, Val loss: 0.829875111579895
Epoch: 24, Train loss: 0.7957280278205872, Val loss: 0.8307376503944397
Epoch: 26, Train loss: 0.7950201630592346, Val loss: 0.8311999440193176
Epoch: 28, Train loss: 0.7940689325332642, Val loss: 0.8298478722572327
E

In [329]:
def add_distance_encoding(data):
    G = pyg_to_networkx_with_attributes(data)
    num_nodes = data.num_nodes

    # Initialize the distance matrix with infinity
    distance_matrix = [[float('inf')] * num_nodes for _ in range(num_nodes)]
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    
    # Populate the distance matrix with actual shortest path lengths
    for i in range(num_nodes):
        distance_matrix[i][i] = 0  # Distance to self is 0
        if i in shortest_paths:
            for j, d in shortest_paths[i].items():
                distance_matrix[i][j] = d

    # Convert the distance matrix to a tensor
    distance_tensor = torch.tensor(distance_matrix, dtype=torch.float)
    
    # Example: Add average distance to node features
    finite_distances = torch.where(distance_tensor == float('inf'), torch.tensor(float('nan')), distance_tensor)
    average_distance = torch.nanmean(finite_distances, dim=1).view(-1, 1)  # Use nanmean to ignore infinities
    data.x = torch.cat([data.x, average_distance], dim=1)
    
    return data

In [330]:
qm9_DE = []
DE_set = copy.deepcopy(qm9_dataset)
for data in DE_set:  # Process 100 molecules as an example
    modified_data = add_distance_encoding(data)
    qm9_DE.append(modified_data)

In [331]:
qm9_DE_dataset = CustomQM9Dataset(qm9_DE)

In [332]:
# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_DE_dataset.data.y[0:train_index].mean()
data_std = qm9_DE_dataset.data.y[0:train_index].std()

qm9_DE_dataset.data.y = (qm9_DE_dataset.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_DE_dataset[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_DE_dataset[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_DE_dataset[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_DE_dataset[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_DE_dataset[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [333]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_DE_dataset[0].x.shape[1]
dim_h = 64
edge_attr = qm9_DE_dataset[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_DE.pt"
)

Epoch: 2, Train loss: 0.805455207824707, Val loss: 0.8395050168037415
Epoch: 4, Train loss: 0.7989227175712585, Val loss: 0.835909366607666
Epoch: 6, Train loss: 0.7978875041007996, Val loss: 0.8300177454948425
Epoch: 8, Train loss: 0.7981917262077332, Val loss: 0.8379428386688232
Epoch: 10, Train loss: 0.7997327446937561, Val loss: 0.8329315185546875
Epoch: 12, Train loss: 0.7966440916061401, Val loss: 0.8342440128326416
Epoch: 14, Train loss: 0.7961634993553162, Val loss: 0.8330127596855164
Epoch: 16, Train loss: 0.7968370914459229, Val loss: 0.8309316635131836
Epoch: 18, Train loss: 0.7977918386459351, Val loss: 0.8349006175994873
Epoch: 20, Train loss: 0.7949866652488708, Val loss: 0.8292779922485352
Epoch: 22, Train loss: 0.7959179878234863, Val loss: 0.8339409828186035
Epoch: 24, Train loss: 0.7962321639060974, Val loss: 0.8338037729263306
Epoch: 26, Train loss: 0.7971914410591125, Val loss: 0.8290377855300903
Epoch: 28, Train loss: 0.7944954633712769, Val loss: 0.828944921493530

In [336]:
from torch_geometric.transforms import AddLaplacianEigenvectorPE

qm9_GE = copy.deepcopy(qm9_dataset)

transform = AddLaplacianEigenvectorPE(k=2, attr_name = None)
qm9_GE.transform = transform

# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_GE.data.y[0:train_index].mean()
data_std = qm9_GE.data.y[0:train_index].std()

qm9_GE.data.y = (qm9_GE.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_GE[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_GE[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_GE[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_GE[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_GE[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

/home/lizhifei/.local/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [337]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_GE[0].x.shape[1]
dim_h = 64
edge_attr = qm9_GE[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_GE.pt"
)

Epoch: 2, Train loss: 0.7951242327690125, Val loss: 0.8293468356132507
Epoch: 4, Train loss: 0.7965684533119202, Val loss: 0.8285439014434814
Epoch: 6, Train loss: 0.7910226583480835, Val loss: 0.8341351747512817
Epoch: 8, Train loss: 0.7912823557853699, Val loss: 0.8266131281852722


KeyboardInterrupt: 

In [341]:
import copy
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import GRU, Linear, ReLU, Sequential

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.nn import GINConv, global_add_pool
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.loader import DataLoader
from torch_geometric.utils import remove_self_loops

target = 0
dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyTransform:
    def __call__(self, data):
        data = copy.copy(data)
        data.y = data.y[:, target]  # Specify target.
        return data


class Complete:
    def __call__(self, data):
        data = copy.copy(data)
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
dataset = QM9(root='dataset/QM9', transform=transform).shuffle()
qm9_dataset = dataset[:5000]

# normalizing the data
mean = qm9_dataset.data.y[1000:5000].mean()
std = qm9_dataset.data.y[1000:5000].std()
qm9_dataset.data.y = (qm9_dataset.data.y - mean) / std

# Split datasets.
test_dataset = dataset[:500]
val_dataset = dataset[500:1000]
train_dataset = dataset[1000:5000]
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [342]:
from torch_geometric.nn import NNConv, Set2Set

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        return out.view(-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(model(data), data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
    return error / len(loader.dataset)


best_val_error = None
for epoch in range(1, 101):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print(f'Epoch: {epoch:03d}, LR: {lr:7f}, Loss: {loss:.7f}, '
          f'Val MAE: {val_error:.7f}, Test MAE: {test_error:.7f}')

Epoch: 001, LR: 0.001000, Loss: 0.0081864, Val MAE: 12.6128933, Test MAE: 13.2576230
Epoch: 002, LR: 0.001000, Loss: 0.0000360, Val MAE: 9.9032644, Test MAE: 9.6857236
Epoch: 003, LR: 0.001000, Loss: 0.0000047, Val MAE: 4.5216296, Test MAE: 4.3478125
Epoch: 004, LR: 0.001000, Loss: 0.0000025, Val MAE: 4.2012156, Test MAE: 4.1506926
Epoch: 005, LR: 0.001000, Loss: 0.0000019, Val MAE: 3.6806682, Test MAE: 3.6976677
Epoch: 006, LR: 0.001000, Loss: 0.0000017, Val MAE: 3.6606716, Test MAE: 3.6427783
Epoch: 007, LR: 0.001000, Loss: 0.0000014, Val MAE: 3.2824202, Test MAE: 3.3308196
Epoch: 008, LR: 0.001000, Loss: 0.0000013, Val MAE: 3.0237275, Test MAE: 3.0077478
Epoch: 009, LR: 0.001000, Loss: 0.0000011, Val MAE: 2.8914763, Test MAE: 2.9436387
Epoch: 010, LR: 0.001000, Loss: 0.0000010, Val MAE: 2.6621523, Test MAE: 2.7135486
Epoch: 011, LR: 0.001000, Loss: 0.0000009, Val MAE: 2.7388948, Test MAE: 2.7135486
Epoch: 012, LR: 0.001000, Loss: 0.0000008, Val MAE: 2.4162644, Test MAE: 2.4829402
Ep

KeyboardInterrupt: 

In [343]:
import copy
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import GRU, Linear, ReLU, Sequential

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.nn import GINConv, global_add_pool
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.loader import DataLoader
from torch_geometric.utils import remove_self_loops

target = 0
dim = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyTransform:
    def __call__(self, data):
        data = copy.copy(data)
        data.y = data.y[:, target]  # Specify target.
        return data


class Complete:
    def __call__(self, data):
        data = copy.copy(data)
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
dataset = QM9(root='dataset/QM9', transform=transform).shuffle()
qm9_dataset = dataset[:5000]

In [346]:
qm9_dataset[0]

Data(x=[16, 11], edge_index=[2, 240], edge_attr=[240, 5], y=[1], pos=[16, 3], z=[16], smiles='[H]N([H])[C@H]1[NH2+]C([H])([H])[C@]([H])(OC([H])([H])[H])O1', name='gdb_11407', idx=[1])

In [344]:
from torch_geometric.transforms import AddLaplacianEigenvectorPE

qm9_GE = copy.deepcopy(qm9_dataset)

transform = AddLaplacianEigenvectorPE(k=2, attr_name = None)
qm9_GE.transform = transform

# data split
data_size = 5000
train_index = int(data_size * 0.8)
test_index = train_index + int(data_size * 0.1)
val_index = test_index + int(data_size * 0.1)


# normalizing the data
data_mean = qm9_GE.data.y[0:train_index].mean()
data_std = qm9_GE.data.y[0:train_index].std()

qm9_GE.data.y = (qm9_GE.data.y - data_mean) / data_std

# datasets into DataLoader
train_loader = DataLoader(qm9_GE[0:train_index], batch_size=64, shuffle=True)
test_loader = DataLoader(qm9_GE[train_index:test_index], batch_size=64, shuffle=True)
val_loader = DataLoader(qm9_GE[test_index:val_index], batch_size=64, shuffle=True)

from torch_geometric.utils import degree
# Determine the maximum possible degree, ensuring it's an integer
max_degree = int(max([degree(data.edge_index[1], num_nodes=data.num_nodes).max().item() for data in qm9_GE[:train_index]]))
deg = torch.zeros(max_degree + 1, dtype=torch.long)

# Compute the in-degree histogram over the training data
for data in qm9_GE[:train_index]:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

In [345]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_features = qm9_GE[0].x.shape[1]
dim_h = 64
edge_attr = qm9_GE[0].edge_attr.shape[1]

model = PNANet(num_features, dim_h, edge_attr, aggregators, scalers, deg).to(device)

epochs = 101

# Remember to change the path if you want to keep the previously trained model
pna_best_loss, pna_train_loss, pna_val_loss = train_epochs(
    epochs, model, train_loader, val_loader, "PNA_0_model_GE.pt"
)

RuntimeError: shape '[64, 1]' is invalid for input of size 1216